#code for compare base model, residual model, residual+bottenlock model

#import module

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import tensorflow as tf
import torchvision
import torchvision.transforms as transforms

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# three blocks for compare

In [2]:
class Residual_Block(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim):
        super(Residual_Block,self).__init__()
        # Residual Block
        self.residual_block = nn.Sequential(
                nn.Conv2d(in_dim, mid_dim, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(mid_dim, out_dim, kernel_size=3, padding=1),
            )            
        self.relu = nn.ReLU()
                  
    def forward(self, x):
        out = self. residual_block(x)  # F(x)
        out = out + x  # F(x) + x
        out = self.relu(out)
        return out

In [17]:
class Residual_Block_with_bottenlock(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim):
        super(Residual_Block_with_bottenlock,self).__init__()
        # Residual Block
        self.residual_block = nn.Sequential(
                nn.Conv2d(in_dim, mid_dim, kernel_size=1),
                nn.ReLU(),
                nn.Conv2d(mid_dim,mid_dim, kernel_size=3, padding=1),
                #nn.ReLu(),
                nn.Conv2d(mid_dim, out_dim, kernel_size=1),
            )            
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self. residual_block(x)  # F(x)
        out = out + x  # F(x) + x
        out = self.relu(out)
        return out

In [4]:
class Base_Block(nn.Module):
    def __init__(self, in_dim, mid_dim, out_dim):
        super(Base_Block,self).__init__()
        # Residual Block
        self.residual_block = nn.Sequential(
                nn.Conv2d(in_dim, mid_dim, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(mid_dim, out_dim, kernel_size=3, padding=1),
            )            
        self.relu = nn.ReLU()
                  
    def forward(self, x):
        out = self. residual_block(x)  # F(x)
        out = out  # F(x)
        out = self.relu(out)
        return out

#build model

In [21]:
class Base_Model(nn.Module):
  def __init__(self,selected_block):
    super(Base_Model,self).__init__()
    if(selected_block=="base"):
      self.layer1 = Base_Block(1,4,4) #28x28 ->28x28
      self.layer2 = Base_Block(4,4,4) #14x14 -> 14x14
      self.layer3 = Base_Block(4,4,4) #7x7 -> 7x7
      self.layer4 = Base_Block(4,4,4) #3x3 -> 3x3
    elif(selected_block=='residual'):
      self.layer1 = Residual_Block(1,4,4)
      self.layer2 = Residual_Block(4,4,4)
      self.layer3 = Residual_Block(4,4,4)
      self.layer4 = Residual_Block(4,4,4)
    else:
      self.layer1 = Residual_Block_with_bottenlock(1,4,4)
      self.layer2 = Residual_Block_with_bottenlock(4,4,4)
      self.layer3 = Residual_Block_with_bottenlock(4,4,4)
      self.layer4 = Residual_Block_with_bottenlock(4,4,4)
    
    self.last_layer = nn.Linear(36,10)
    
  def forward(self, x):
      out = self.layer1(x)
      #print(out.size())
      out = nn.AvgPool2d(2)(out)
      #print(out.size())
      out = self.layer2(out)
      #print(out.size())
      out = nn.AvgPool2d(2)(out)
      #print(out.size())
      out = self.layer3(out)
      #print(out.size())
      out = nn.AvgPool2d(2)(out)
      #print(out.size())
      out = self.layer4(out)
      #print(out.size())
      out = out.view(16,-1) #64 is batch_Size
      out = self.last_layer(out)
      return out

#load data

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

In [7]:
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)

#see result

In [8]:
def eval_model(model,criterion,optimizer):
  num_epochs=5
  for epoch in range(num_epochs):
      # Train the model for one epoch
      model.train()
      train_loss = 0.0
      train_correct = 0
      train_total = 0
      for inputs, labels in trainloader:
          inputs, labels = inputs.to(device), labels.to(device)
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # Update the training loss and accuracy
          train_loss += loss.item() * inputs.size(0)
          _, predicted = torch.max(outputs.data, 1)
          train_correct += (predicted == labels).sum().item()
          train_total += labels.size(0)

      # Compute the validation accuracy
      model.eval()
      val_correct = 0
      val_total = 0
      with torch.no_grad():
          for inputs, labels in testloader:
              inputs, labels = inputs.to(device), labels.to(device)
              outputs = model(inputs)
              _, predicted = torch.max(outputs.data, 1)
              val_correct += (predicted == labels).sum().item()
              val_total += labels.size(0)
      val_acc = val_correct / val_total

      # Print the training loss and validation accuracy for this epoch
      train_loss = train_loss / len(trainloader.dataset)
      train_acc = train_correct / train_total
      print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} Val Acc: {val_acc:.4f}')

In [33]:
import time
model = Base_Model('base').to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

start = time.time()
eval_model(model,criterion,optimizer)
fin = time.time()
print(f"total time: {fin-start:.2f} second")

Epoch 1/5: Train Loss: 2.3018 Train Acc: 0.1114 Val Acc: 0.1135
Epoch 2/5: Train Loss: 2.3014 Train Acc: 0.1124 Val Acc: 0.1135
Epoch 3/5: Train Loss: 2.3014 Train Acc: 0.1124 Val Acc: 0.1135
Epoch 4/5: Train Loss: 2.3014 Train Acc: 0.1124 Val Acc: 0.1135
Epoch 5/5: Train Loss: 2.3013 Train Acc: 0.1124 Val Acc: 0.1135
total time: 205.11 second


In [32]:
import time
model = Base_Model('residual').to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

start = time.time()
eval_model(model,criterion,optimizer)
fin = time.time()
print(f"total time: {fin-start:.2f} second")

Epoch 1/5: Train Loss: 0.9464 Train Acc: 0.6869 Val Acc: 0.9262
Epoch 2/5: Train Loss: 0.2057 Train Acc: 0.9369 Val Acc: 0.9536
Epoch 3/5: Train Loss: 0.1421 Train Acc: 0.9563 Val Acc: 0.9639
Epoch 4/5: Train Loss: 0.1120 Train Acc: 0.9647 Val Acc: 0.9683
Epoch 5/5: Train Loss: 0.0995 Train Acc: 0.9684 Val Acc: 0.9742
total time: 211.52 second


In [22]:
import time
model = Base_Model('residual_with_bottenlock').to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

start = time.time()
eval_model(model,criterion,optimizer)
fin = time.time()
print(f"total time: {fin-start:.2f} second")

Epoch 1/5: Train Loss: 1.1405 Train Acc: 0.6259 Val Acc: 0.8550
Epoch 2/5: Train Loss: 0.4335 Train Acc: 0.8647 Val Acc: 0.8951
Epoch 3/5: Train Loss: 0.3531 Train Acc: 0.8928 Val Acc: 0.9051
Epoch 4/5: Train Loss: 0.3136 Train Acc: 0.9019 Val Acc: 0.9034
Epoch 5/5: Train Loss: 0.2917 Train Acc: 0.9077 Val Acc: 0.9148
total time: 223.22 second


In [28]:
model = Base_Model('base').to(device)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 1446


In [29]:
model = Base_Model('residual').to(device)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 1446


In [27]:
model = Base_Model('residual_with_bottenlock').to(device)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 1110


residual구조가 없는 base model은 8개라는 적은 층에도 불구하고 학습이 이루어지지 않음.

모델이 작기 때문에 bottenlock 구조와 residual구조의 파라미터 개수 차이가 적어 유의미한 속도 차이가 없는 것으로 보임